In [1]:
#getting data for observed, expected, chi squared
import pandas as pd
df = pd.read_csv('GBR_data.tsv', sep = '\t')

In [2]:
# Calculating observed and expected

def h_exp_obs_Aa(AA, Aa, aa):
    
    'Calculates expected and observed heterzygosity'
    
    p = ((2 * AA) + Aa)/ (2 * (AA + Aa + aa))
    q = ((2 * aa) + Aa)/ (2 * (AA + Aa + aa))
    
    #2pq
    twice_p_q = 2 * p * q
    
    #Calculating Observed
    Observed = (Aa *2) /(2 * (AA + Aa + aa))
 
    return Observed, twice_p_q

In [3]:
### Calculating Chi squared
### Testing for Hardy-Weinberg Equilibrium
### Null hyp --> No difference between the observed and expected data

from scipy.stats import chisquare

def H_W_Equlibrium(AA, Aa, aa):
    
    'Calculates whether there is a significant difference between the observed and expected data.'
    
    data = [AA, Aa, aa]
    chi, p= chisquare(data)
    return p

In [4]:
#calculating where there is a difference between observed and expected
# note it returns nan when values are nearly the same

from scipy.stats import bartlett

def equal_variance(O, E):     
    stat, p = bartlett(O, E)
    return p


In [ ]:
#Observed and expected heterzygosity & chi squared

hom_ref =[] #12:14        # didnt know how to iterate using dictionary so used indices
het = [] # 22:24
hom_alt = [] #37:38
rs = [] # rs values

GBR_geno = df['genotypes']
for x in GBR_geno:
    hom_ref.append(x[12:14])
    het.append(x[22:24])
    hom_alt.append(x[37:38])
    
GBR_rs = df['ID']
for x in GBR_rs:
    rs.append(x)
    
GBR_data = list(zip(hom_ref, het, hom_alt))  #getting the data
GBR = list(zip(rs, GBR_data))
#print(GBR)

rs_value = []
C = [] #observed and expected
E = [] #H_W

for x in GBR:
    try:
        rs_value.append(x[0])
        Calculations = h_exp_obs_Aa(AA = int(x[1][0]), Aa = int(x[1][1]), aa = int(x[1][2]))
        C.append(Calculations)
        Equ = H_W_Equlibrium(AA = int(x[1][0]), Aa = int(x[1][1]), aa = int(x[1][2]))
        E.append(Equ)
    except:
        Calculations = 'N/A' 
        Equ = 'N/A'
          
C.append(Calculations)
O_E = list(zip(rs_value, C, E, ))


#outputing tsv
import csv    

with open('GBR_O_E_CHI_.tsv','w', newline='') as f_output:
    tsv_output = csv.writer(f_output, delimiter='\t')
    tsv_output.writerow(O_E)



In [5]:
##FST eq for just one population

#if mutiple populations are selected then population genetic variation (FST value) for each pair of populations should be reported

def Fst(population, pop_AA, pop_Aa, pop_aa):
    
    p = ((2 *pop_AA ) + pop_Aa) / (2 * population)    #calculating p and q values
    q= 1 - p
    
    H_obs = pop_Aa / population
    
    H_exp = 2 * p * q
    
    F = (H_exp - H_obs ) /H_exp
    
    p_bar= ((2* pop_AA ) + pop_Aa )/ (2 *(population))
    q_bar = ((2 *pop_aa ) + pop_Aa) /  (2 *(population))
    
    HI = (H_obs * population) / population
    HS= (H_exp * population) / population
    
    HT = 2 * p_bar * q_bar
    F_ST = (HT - HS)/ HT
    
    return F_ST
    
                                      
    

In [6]:
####FINAL fST eqn
#http://www.uwyo.edu/dbmcd/molmark/practica/fst.html
#N = number of populations
#hom_ref = AA       
#het = Aa
#hom_alt = aa
#population_ = number of samples

def Fst(N,population_1, population_2, 
             pop1_AA, pop2_AA,
             pop1_Aa, pop2_Aa,
             pop1_aa, pop2_aa,
             population_3 = 0, population_4 = 0, population_5 = 0, 
             pop3_AA = 0, pop4_AA = 0, pop5_AA = 0,
             pop3_Aa = 0, pop4_Aa = 0, pop5_Aa = 0,
             pop3_aa = 0, pop4_aa = 0, pop5_aa = 0):
    
    'Calculates F-statistics from genotypic data.'
    
    if N == 2:
        p_1 = ((2 *pop1_AA ) + pop1_Aa) / (2 * population_1)    #calculating p and q values
        q_1= 1 - p_1
    
        p_2 = ((2* pop2_AA ) + pop2_Aa) / (2 * population_2)
        q_2= 1 - p_2
        
        H_obs1 = pop1_Aa / population_1  #calculating observed heterzygosity
        H_obs2 = pop2_Aa / population_2
        
        H_exp_1 = 2 * p_1 * q_1    #calculating expected heterzygosity(2pq)
        H_exp_2 = 2 * p_2 * q_2
        
        F_1 = (H_exp_1 - H_obs1 ) /H_exp_1  #calculating population inbreeding coefficient
        F_2 = (H_exp_2 - H_obs2 ) /H_exp_2
        
        #calculating p-bar and q-bar
        
        p_bar= ((2* pop1_AA ) + pop1_Aa + (2* pop2_AA ) + pop2_Aa)/ (2 *(population_1 + population_2))
        q_bar = ((2 *pop1_aa ) + pop1_Aa + (2 *pop2_aa ) + pop2_Aa) /  (2 *(population_1 + population_2))
       
        #HI based on observed heterozygosities in populations
        HI = ((H_obs1 * population_1) + (H_obs2 * population_2)) / (population_1 + population_2)
     
        #HS based on expected heterozygosities in populations
        HS= (H_exp_1 * population_1) + (H_exp_2 * population_2) / (population_1 + population_2)
        
        #HT based on expected heterozygosities overall:
        HT = 2 * p_bar * q_bar
        
        
    elif N == 3:
        p_1 = ((2 *pop1_AA ) + pop1_Aa) / (2 * population_1)
        q_1= 1 - p_1
    
        p_2 = ((2* pop2_AA ) + pop2_Aa) / (2 * population_2)
        q_2= 1 - p_2
        
        p_3 = ((2* pop3_AA ) + pop3_Aa) / (2 * population_3)
        q_3 = 1 - p_3
        
        H_obs1 = pop1_Aa / population_1
        H_obs2 = pop2_Aa / population_2
        H_obs3 = pop3_Aa / population_3
        
        H_exp_1 = 2 * p_1 * q_1
        H_exp_2 = 2 * p_2 * q_2
        H_exp_3 = 2 * p_3 * q_3
        
        F_1 = (H_exp_1 - H_obs1 ) /H_exp_1
        F_2 = (H_exp_2 - H_obs2 ) /H_exp_2
        F_3 = (H_exp_3 - H_obs3 ) /H_exp_3
        
        p_bar = ((2* pop1_AA ) + pop1_Aa + (2* pop2_AA ) + pop2_Aa + (2* pop3_AA ) + pop3_Aa) / (2 *(population_1 + population_2 +population_3))
        q_bar = ((2 *pop1_aa ) + pop1_Aa + (2 *pop2_aa ) + pop2_Aa + (2 *pop3_aa ) + pop3_Aa) / (2 *(population_1 + population_2 + population_3))
        
        HI= ((H_obs1 * population_1) + (H_obs2 * population_2) + (H_obs3 * population_3 )) / (population_1 + population_2 + population_3)
    
        HS = ((H_exp_1 * population_1) + (H_exp_2 * population_2) + (H_exp_3 * population_3 )) / (population_1 + population_2 + population_3)
        
        HT = 2 * p_bar * q_bar
        
        
    elif N == 4:
        p_1 = ((2 *pop1_AA ) + pop1_Aa) / (2 * population_1)
        q_1= 1 - p_1
    
        p_2 = ((2* pop2_AA ) + pop2_Aa) / (2 * population_2)
        q_2= 1 - p_2
        
        p_3 = ((2* pop3_AA ) + pop3_Aa) / (2 * population_3)
        q_3 = 1 - p_3
        
        p_4 = ((2* pop4_AA ) + pop4_Aa) / (2 * population_4)
        q_4 = 1 - p_4
        
        H_obs1 = pop1_Aa / population_1
        H_obs2 = pop2_Aa / population_2
        H_obs3 = pop3_Aa / population_3
        H_obs4 = pop4_Aa / population_4
        
        H_exp_1 = 2 * p_1 * q_1
        H_exp_2 = 2 * p_2 * q_2
        H_exp_3 = 2 * p_3 * q_3
        H_exp_4 = 2 * p_4 * q_4
        
        F_1 = (H_exp_1 - H_obs1 ) /H_exp_1
        F_2 = (H_exp_2 - H_obs2 ) /H_exp_2
        F_3 = (H_exp_3 - H_obs3 ) /H_exp_3
        F_4 = (H_exp_4 - H_obs4 ) /H_exp_4
        
        p_bar= ((2* pop1_AA ) + pop1_Aa + (2* pop2_AA ) + pop2_Aa + (2* pop3_AA ) + pop3_Aa + (2* pop4_AA ) + pop4_Aa)/ (2 *(population_1 + population_2 +population_3 +population_4))
        q_bar= ((2 *pop1_aa ) + pop1_Aa + (2 *pop2_aa ) + pop2_Aa + (2 *pop3_aa ) + pop3_Aa +  (2 *pop4_aa ) + pop4_Aa) / (2 *(population_1 + population_2 + population_3 + population_4 ))
        
        HI= ((H_obs1 * population_1) + (H_obs2 * population_2) + (H_obs3 * population_3 ) + (H_obs4 * population_4 )) / (population_1 + population_2 + population_3 + population_4)
    
        HS= ((H_exp_1 * population_1) + (H_exp_2 * population_2) + (H_exp_3 * population_3 ) + (H_exp_4 * population_4 )) / (population_1 + population_2 + population_3 + population_4)
        
        HT = 2 * p_bar * q_bar
        
    else:
        p_1 = ((2 *pop1_AA ) + pop1_Aa) / (2 * population_1)
        q_1= 1 - p_1
    
        p_2 = ((2* pop2_AA ) + pop2_Aa) / (2 * population_2)
        q_2= 1 - p_2
        
        p_3 = ((2* pop3_AA ) + pop3_Aa) / (2 * population_3)
        q_3 = 1 - p_3
        
        p_4 = ((2* pop4_AA ) + pop4_Aa) / (2 * population_4)
        q_4 = 1 - p_4
        
        p_5 = ((2* pop5_AA ) + pop5_Aa) / (2 * population_5)
        q_5 = 1 - p_5
        
        H_obs1 = pop1_Aa / population_1
        H_obs2 = pop2_Aa / population_2
        H_obs3 = pop3_Aa / population_3
        H_obs4 = pop4_Aa / population_4
        H_obs5 = pop5_Aa / population_5
        
        H_exp_1 = 2 * p_1 * q_1
        H_exp_2 = 2 * p_2 * q_2
        H_exp_3 = 2 * p_3 * q_3
        H_exp_4 = 2 * p_4 * q_4
        H_exp_5 = 2 * p_5 * q_5
        
        F_1 = (H_exp_1 - H_obs1 ) /H_exp_1
        F_2 = (H_exp_2 - H_obs2 ) /H_exp_2
        F_3 = (H_exp_3 - H_obs3 ) /H_exp_3
        F_4 = (H_exp_4 - H_obs4 ) /H_exp_4
        F_5 = (H_exp_5 - H_obs5 ) /H_exp_5
        
        p_bar = ((2* pop1_AA ) + pop1_Aa + (2* pop2_AA ) + pop2_Aa + (2* pop3_AA ) + pop3_Aa + (2* pop4_AA ) + pop4_Aa + (2* pop5_AA ) + pop5_Aa)/ (2 *(population_1 + population_2 +population_3 +population_4 + population_5))
        q_bar = ((2 *pop1_aa ) + pop1_Aa + (2 *pop2_aa ) + pop2_Aa + (2 *pop3_aa ) + pop3_Aa +  (2 *pop4_aa ) + pop4_Aa 
        +(2 *pop5_aa ) + pop5_Aa)/ (2 *(population_1 + population_2 + population_3 + population_4 + population_5))
        
        HI = ((H_obs1 * population_1) + (H_obs2 * population_2) + (H_obs3 * population_3 ) + (H_obs4 * population_4 ) 
        + (H_obs5 * population_5 )) / (population_1 + population_2 + population_3 + population_4 + population_5)
    
        HS = ((H_exp_1 * population_1) + (H_exp_2 * population_2) + (H_exp_3 * population_3 ) + (H_exp_4 * population_4 ) 
        + (H_exp_5 * population_5 ))/ (population_1 + population_2 + population_3 + population_4 + population_5)
        
        HT = 2 * p_bar * q_bar
        
    F_IS = (HS - HI) / HS
    F_ST = (HT - HS)/ HT
    F_IT = (HT -HI)/ HT
    
    return F_IS, F_ST, F_IT  # Calculating F-Statistics




In [7]:
# checking with the ones on website
# http://www.uwyo.edu/dbmcd/molmark/practica/fst.html
A = Fst(N = 3, population_1 = 500, population_2 =100, 
             pop1_AA = 125, pop2_AA = 50,
             pop1_Aa = 250, pop2_Aa = 30,
             pop1_aa = 125, pop2_aa =20,
             population_3 = 1000,
             pop3_AA = 100, 
             pop3_Aa = 500,
             pop3_aa = 400)            
          

In [8]:
print(A)

(-0.039307128580946024, 0.03437738731856383, -0.0035784648787744212)


F-statistics: a measure of genetic structure developed by Sewall Wright (1969, 1978). Related to statistical analysis of variance  (ANOVA)
FST is the proportion of the total genetic variance contained in a subpopulation (the S subscript) relative to the total genetic variance (the T subscript). Values can range from 0 to 1. High FST implies a considerable degree of differentiation among populations.
FIS (inbreeding coefficient) is the proportion of the variance in the subpopulation contained in an individual. High FIS implies a considerable degree of inbreeding.


In [9]:
#producing graphs for expected and observed.

import plotly
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd

In [67]:
#practice with dummy data
df = pd.read_csv('dummy_data_1.csv')
#print(df)

In [93]:
#https://plotly.com/python/bar-charts/

import plotly.graph_objects as go

fig = go.Figure(data = [
    go.Scatter(name = 'observed', x = df['ID'], y = df['observed']),
    go.Scatter(name = 'expected', x = df['ID'], y = df['Expected'])
])

fig.show()    



Low heterozygosity means little genetic variability.
If the observed heterozygosity is lower than expected, we seek to attribute the discrepancy to forces such as inbreeding. 
If heterozygosity is higher than expected, we might suspect an isolate-breaking effect (the mixing of two previously isolated populations).